In [ ]:
"""
IMDB emotional analysis using RNN, LSTM
"""

'\nIMDB emotional analysis using RNN, LSTM\n'

In [ ]:
import urllib.request
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f8d83beb9a0>)

In [ ]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [ ]:
#data split
train_df = df[:25000]
test_df = df[25000:]

train_X = train_df['review'].tolist()
train_y = train_df['sentiment'].values

test_X = test_df['review'].tolist()
test_y = test_df['sentiment'].values

print("train X, Y : ", len(train_X), len(train_y) )

train X, Y :  25000 25000


In [ ]:
#tokenize
word_set = set()
for sentence in train_X:
  word_set.update(sentence.split())
vocab = {word: i+2 for i , word in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1
print("vocab count : ", len(vocab))

vocab count :  285121


In [ ]:
embedding_layer = nn.Embedding(num_embeddings = len(vocab), embedding_dim = 15, padding_idx = 1)

In [ ]:
#encoding
train_enc_X = []
test_enc_X = []
for i in train_X:
  tmp = i.split()
  tmp2 = []
  for t in tmp:
    tmp2.append(vocab[t])
  train_enc_X.append(torch.LongTensor(tmp2))
for i in test_X:
  tmp3 = i.split()
  tmp4 = []
  for t in tmp:
    tmp3.append(vocab[t])
  test_enc_X.append(torch.LongTensor(tmp4))
print(len(train_enc_X))

25000


In [ ]:
max_len = 0
for documents in train_enc_X:
  if len(documents) > max_len:
    max_len = len(documents)
for documents in test_enc_X:
  if len(documents) > max_len:
    max_len = len(documents)

print(max_len)
#push padding

for doc in train_enc_X:
  tmp = ['<pad>' for i in range(max_len - len(doc))]
  doc = doc + tmp
for doc in test_enc_X:
  tmp = ['<pad>' for i in range(max_len - len(doc))]
  doc = doc + tmp

for i in range(10):
  print(len(train_enc_X[i]))

2470


TypeError: ignored

In [ ]:
class RNN(nn.Module):
  def __init__(self,input_size, hidden_size, num_layers,num_classes):
    super(RNN,self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.num_classes = num_classes
    
    #Elman Rnn
    self.rnn = nn.RNN(input_size = input_size, hidden_size = hidden_size, 
                      num_layers = num_layers, batch_first = True, bias = True)
    self.relu = nn.ReLU()
    self.fc = nn.Linear(hidden_size , num_classes)

  def forward(self,x):
    #x.size(0) => batch_Size
    h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
    out,_ = self.rnn(x,h_0)
    out = self.fc(out[:][-1][:])
    return out

In [ ]:
#hyperparameter
window_size = 15
hidden_size = 20
num_classes = 1
num_layers = 1

lr = 0.001
num_epochs = 1000
#train_enc_X
#25000 * 133

train_tensor_X = torch.stack(train_enc_X).view(1,-1,25000)
train_tensor_y = torch.Tensor(train_y)

test_tensor_X = torch.stack(test_enc_X).view(1,-1,25000)
test_tensor_y = torch.Tensor(test_y)

RuntimeError: ignored

In [ ]:

print("Size train_X" , train_tensor_X.shape)
print("Size train_y ", train_tensor_y.shape)
print("Size test_X", test_tensor_X.shape)


In [ ]:
model = RNN(window_size,hidden_size, num_layers, num_classes)
optimizer = optim.Adam(model.parameters(), lr = lr)
critertion = torch.nn.MSELoss()
for epoch in range(num_epochs+1):
  output = model.forward(X_train)
  optimizer.zero_grad()
  loss = critertion(output, y_train)
  loss.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print("Epoch : ", epoch, "loss :",  loss.item())